# Przykład dla danych tekstowych (dwie klasy)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nosarzewski/TUS/blob/master/LIME_not_run.ipynb)

W tym przykładzie wykorzystamy zbiór [20 newsgroups text dataset](https://scikit-learn.org/stable/datasets/#the-20-newsgroups-text-dataset).

Zawiera on dane tekstowe dla ponad 18.000 dokumentów, pogrupowanych w 20 bloków tematycznych.

Skorzystamy z wyłącznie dwóch grup: chrześcijaństwo oraz ateizm.

## 1. Wczytanie bibliotek

In [0]:
%%capture
!pip install lime

In [0]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

## 2. Wczytanie danych

In [0]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset = 'train', categories = categories)
newsgroups_test = fetch_20newsgroups(subset = 'test', categories = categories)
class_names = ['atheism', 'christian']

## 3. Budowa modelu
Wykorzystamy las losowy (*aka stochastic jungle*) do predykcji.

Niestety dane tekstowe w surowej postaci nie nadają się do wykorzystania przez model. Dletego do reprezentacji tekstów wykorzystamy [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) (term frequency - inverse term frequency), które wskazuje jak ważne jest dane słowo w dokumencie:

In [0]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase = False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

Następnie wytrenujmy nasz las losowy:

In [0]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators = 500)
rf.fit(train_vectors, newsgroups_train.target)

Skoro mamy już model, to dokonajmy predykcji:

In [0]:
pred = rf.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred)

Nasz model osiąga wysoką wartość F1, co powinno świadczyć o dobrej jakości modelu.

Jednakże, [tutorial sklearn](https://scikit-learn.org/stable/datasets/#filtering-text-for-more-realistic-training) wskazuje, że naiwny klasyfikator bayesowski jest przeuczony w stosunku do danych i wychwytuje zależności, które nie mają uzasadnienia. Sprawdźmy, czy dla dla naszego lasu losowego także jest to prawda.

## 4. Wyjaśnianie modelu z wykorzystaniem LIME

Nasz model został wyuczony na wartościach TF-IDF, podczas gdy LIME działa na samych słowach. 

Wykorzystamy zdolności pakietu sklearn aby obejść tę niedogodność:

In [0]:
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)

In [0]:
print(c.predict_proba([newsgroups_test.data[0]]))

Teraz możemy dokonać wyjaśnienia modelu za pomocą LIME:

In [0]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names = class_names)

Dokonajmy wytłumaczenia dla dokumentu nr 83. Model wykorzystywany do wyjaśniania powinien koszystać z co najwyżej 6 cech (zmiennych).

In [0]:
idx = 83
exp = explainer.explain_instance(newsgroups_test.data[idx], 
                                 c.predict_proba, 
                                 num_features = 6)
print('Document id: %d' % idx)
print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

Sprawdźmy z czego korzystał nasz model podczas prognozowania:

In [0]:
exp.as_list()

Widzimy, że dwa najważniejsze słowa to 'Posting' oraz 'Host'. Jeśli byśmy je usunęli to prawdopodobieństwo tego, że dokument należy do klasy 'christian' powinno wzrosnąć o 27 p.p.

Sprawdźmy czy tak rzeczywiście jest!

In [0]:
print('Original prediction:', rf.predict_proba(test_vectors[idx])[0,1])
tmp = test_vectors[idx].copy()
tmp[0,vectorizer.vocabulary_['Posting']] = 0
tmp[0,vectorizer.vocabulary_['Host']] = 0
print('Prediction removing some features:', rf.predict_proba(tmp)[0,1])
print('Difference:', round(rf.predict_proba(tmp)[0,1] - rf.predict_proba(test_vectors[idx])[0,1], 3))

## 5. Wizualizacja

Spróbujmy zwizualizować nasze wyniki.

In [0]:
%matplotlib inline
fig = exp.as_pyplot_figure()

Możemy też pokazać wpływ słów w sposób bardziej zrozumiały dla laika.

In [0]:
exp.show_in_notebook(text = False)

Możemy także zaznaczyć słowa mające największy wpływ na predykcję w samym oryginalnym dokumencie.

In [0]:
exp.show_in_notebook(text = True)

Widzimy, że choć zdawało się, że nasz model działa dobrze (osiąga wysokie wartości miary F), to robi to w sposób całkowicie nieuzasadniony dla człowieka.